In [170]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")
from millify import millify

In [171]:
from pathlib import Path
p = Path().cwd()
print(f"original File path: {p}")
stringpath= str(p)[0:str(p).rfind('\\')] + '\\data'
print(f"Updated File Path to pick from Data Library: { stringpath}")
ngsimfile=stringpath + '/' + 'Cleaned_NGSIM_Data_Part_1.csv'

ngsim = pd.read_csv(ngsimfile,low_memory=False)
print(f"{ngsim.duplicated().sum()} duplicate values have been removed")
ngsim.drop_duplicates(inplace=True)

original File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
Updated File Path to pick from Data Library: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data
0 duplicate values have been removed


In [172]:
ngsim['Velocity Difference_Following-Preceding'] = np.NaN
ngsim['Acceleration Difference_Following-Preceding'] = np.NaN
filtered_ngsim_U= ngsim[((ngsim['Location'] == 'us-101'))  ]

filtered_ngsim_U= ngsim[((ngsim['Location'] == 'us-101')) & ((ngsim['Vehicle_ID'] <= 100 )  ) & ((ngsim['Preceding'] <= 100 )  )]
filtered_ngsim_I= ngsim[(ngsim['Location'] == 'i-80') ]


In [173]:
filtered_ngsim_U.shape

(154804, 21)

In [174]:
filtered_ngsim_U.set_index(['Vehicle_ID', 'Preceding','Relative_Time'], append=True,inplace=True)
filtered_ngsim_U=filtered_ngsim_U.sort_index()
print(filtered_ngsim_U.index.is_monotonic_increasing)

True


In [175]:
filtered_ngsim_U.index.rename(['id', 'Vehicle_ID', 'Preceding', 'Relative_Time'], inplace=True)
filtered_ngsim_U.index

MultiIndex([(     11,  5,  0, 1117684841001),
            (     44, 52, 48, 1117685817401),
            (     51, 73, 67, 1117684928901),
            (     91,  6,  2, 1117685787801),
            (    112, 41, 35, 1117684872901),
            (    122, 77, 72, 1117685822701),
            (    133, 14, 10, 1117684889001),
            (    139, 73, 63, 1117684012201),
            (    176, 26, 22, 1117683969901),
            (    189, 62, 58, 1117684893801),
            ...
            (4098616, 91, 86, 1117685859701),
            (4098632, 83, 67, 1117684011101),
            (4098637, 11,  3, 1117685766501),
            (4098661, 74, 73, 1117685787201),
            (4098664, 85, 80, 1117684887001),
            (4098686, 14, 11, 1117685780801),
            (4098797, 83, 76, 1117683997501),
            (4098811, 83, 90, 1117684899701),
            (4098864, 39, 32, 1117684866701),
            (4098873, 75, 68, 1117685841201)],
           names=['id', 'Vehicle_ID', 'Preceding', 'Relative_Ti

In [176]:
velocity_diff=[]
acceleration_diff=[]

for i in filtered_ngsim_U.index.get_level_values(0):
    if i%10000==0:
        print(f"row count{millify(i, precision=2, drop_nulls=False)} processed")
    previous_Vehicle_ID=filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == i )].index.get_level_values("Preceding").values[0]
    current_time=filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == i )].index.get_level_values("Relative_Time")[0]
    #print(f"i:{i}")
    #print(f"previous_Vehicle_ID: {previous_Vehicle_ID}")
    #print(f"current_time: {current_time}")
    if previous_Vehicle_ID > 0:
        preceding_corresponding_ix=filtered_ngsim_U.xs((current_time,previous_Vehicle_ID),level=("Relative_Time",'Vehicle_ID')).index.get_level_values('id').values[0] 
        velocity = filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == i ) ].values - filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == preceding_corresponding_ix ) ].values
        acceleration = filtered_ngsim_U['v_Acc'][(filtered_ngsim_U.index.get_level_values('id') == i ) ].values - filtered_ngsim_U['v_Acc'][(filtered_ngsim_U.index.get_level_values('id') == preceding_corresponding_ix ) ].values
    else:
        velocity = filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == i ) ].values - 0
        acceleration = filtered_ngsim_U['v_Acc'][(filtered_ngsim_U.index.get_level_values('id') == i ) ].values - 0
    #print(preceding_corresponding_ix)
    #print(f"velocity previous :{ filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == preceding_corresponding_id ) ].values}")
    #print(f"velocity current : { filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == i ) ].values}")
#    if i >= 1000:
#        break    
    velocity_diff.append(velocity[0])
    acceleration_diff.append(acceleration[0])

row count90.00k processed
row count360.00k processed
row count510.00k processed
row count1.69M processed
row count1.94M processed
row count3.15M processed
row count3.36M processed
row count3.67M processed


In [177]:
filtered_ngsim_U['Velocity Difference_Following-Preceding'] = velocity_diff
filtered_ngsim_U['Acceleration Difference_Following-Preceding'] = acceleration_diff

In [178]:
#filtered_ngsim_U.xs((1117685909701,242),level=("Relative_Time",'Vehicle_ID')).index.get_level_values('id').values[0] 

In [179]:
'''
i=546
previous_Vehicle_ID=filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == i )].index.get_level_values("Preceding").values[0]
current_time=filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == i )].index.get_level_values("Relative_Time")[0]
print(f"i:{i}")
print(f"previous_Vehicle_ID: {previous_Vehicle_ID}")
print(f"current_time: {current_time}")
if previous_Vehicle_ID > 0:
    preceding_corresponding_ix=filtered_ngsim_U.xs((current_time,previous_Vehicle_ID),level=("Relative_Time",'Vehicle_ID')).index.get_level_values('id').values[0] 
    velocity = filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == i ) ].values - filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == preceding_corresponding_ix ) ].values
else:
    velocity = filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == i ) ].values - 0

'''


'\ni=546\nprevious_Vehicle_ID=filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == i )].index.get_level_values("Preceding").values[0]\ncurrent_time=filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == i )].index.get_level_values("Relative_Time")[0]\nprint(f"i:{i}")\nprint(f"previous_Vehicle_ID: {previous_Vehicle_ID}")\nprint(f"current_time: {current_time}")\nif previous_Vehicle_ID > 0:\n    preceding_corresponding_ix=filtered_ngsim_U.xs((current_time,previous_Vehicle_ID),level=("Relative_Time",\'Vehicle_ID\')).index.get_level_values(\'id\').values[0] \n    velocity = filtered_ngsim_U[\'v_Vel\'][(filtered_ngsim_U.index.get_level_values(\'id\') == i ) ].values - filtered_ngsim_U[\'v_Vel\'][(filtered_ngsim_U.index.get_level_values(\'id\') == preceding_corresponding_ix ) ].values\nelse:\n    velocity = filtered_ngsim_U[\'v_Vel\'][(filtered_ngsim_U.index.get_level_values(\'id\') == i ) ].values - 0\n\n'

In [180]:
'''
filtered_ngsim_U.xs((filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 546 )].index.get_level_values("Relative_Time")[0],filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 546 )].index.get_level_values("Preceding").values[0]),level=("Relative_Time",'Vehicle_ID')).index.get_level_values('id').values[0] 
ngsim[(ngsim['Vehicle_ID'] == 242)& (ngsim['Relative_Time'] == 1117685909701) ]
filtered_ngsim_U.xs((filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 0 )].index.get_level_values("Relative_Time")[0],filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 0 )].index.get_level_values("Preceding").values[0]),level=("Relative_Time",'Vehicle_ID')).index.get_level_values('id').values[0] 
filtered_ngsim_U.index.get_level_values("Preceding")
ngsim[(ngsim['Vehicle_ID'] == 242)|(ngsim['Preceding'] == 242) ].to_csv('ng', index = True)
filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("Vehicle_ID") == 242 )].sort_values("Relative_Time").to_csv('test', index = True)
                            #1117685909701
'''

'\nfiltered_ngsim_U.xs((filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 546 )].index.get_level_values("Relative_Time")[0],filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 546 )].index.get_level_values("Preceding").values[0]),level=("Relative_Time",\'Vehicle_ID\')).index.get_level_values(\'id\').values[0] \nngsim[(ngsim[\'Vehicle_ID\'] == 242)& (ngsim[\'Relative_Time\'] == 1117685909701) ]\nfiltered_ngsim_U.xs((filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 0 )].index.get_level_values("Relative_Time")[0],filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 0 )].index.get_level_values("Preceding").values[0]),level=("Relative_Time",\'Vehicle_ID\')).index.get_level_values(\'id\').values[0] \nfiltered_ngsim_U.index.get_level_values("Preceding")\nngsim[(ngsim[\'Vehicle_ID\'] == 242)|(ngsim[\'Preceding\'] == 242) ].to_csv(\'ng\', index = True)\nfiltered_ngsim_U[(filtered_ngsim_U.index.get_level_values("Vehicle_ID") == 242 

In [181]:
filtered_ngsim= filtered_ngsim_U

In [182]:
stringpath= str(p)[0:str(p).rfind('\\')] + '\\data'
ngsimfilteredfile=stringpath + '\\' + 'Sample_Cleaned_NGSIM_Data_Part_2.csv'
filtered_ngsim.to_csv(ngsimfilteredfile, index = True)